<a href="https://colab.research.google.com/github/samarthya04/IIIT-A-internship/blob/main/ChessGamePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
from google.colab  import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
path = '/content/drive/MyDrive/IIIT-A Internship/games.csv'
df = pd.read_csv(path)

In [22]:
df.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  object 
 15  opening_ply     20058 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 2.3+ MB


# Preprocessing

In [24]:
df['winner'] = df['winner'].map({'white': 1, 'black': 0, 'draw': 2})
df['rating_difference'] = abs(df['white_rating'] - df['black_rating'])
df = pd.get_dummies(df, columns=['victory_status'])

In [25]:
features = ['winner', 'white_rating', 'black_rating', 'rating_difference', 'increment_code']
features += [cols for cols in df.columns if cols.startswith('victory_status_')]
df = df[features]

In [26]:
X = df.drop(columns=['winner'])
y = df['winner']

In [27]:
from sklearn.preprocessing import *
label_encoder = LabelEncoder()
X['increment_code'] = label_encoder.fit_transform(X['increment_code'])

# Logistic Regression

In [28]:
scaler = StandardScaler()
X_stand = scaler.fit_transform(X)

In [31]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_stand)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=1)

In [33]:
logr = LogisticRegression()
logr.fit(X_train, y_train)

LogisticRegression()

In [34]:
y_pred = logr.predict(X_test)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.09      0.15      2747
           1       0.53      0.93      0.67      2996
           2       1.00      0.96      0.98       275

    accuracy                           0.54      6018
   macro avg       0.68      0.66      0.60      6018
weighted avg       0.54      0.54      0.45      6018



# Random Forest Classifier

In [16]:
minmax_scaler = MinMaxScaler()
X_minmax = minmax_scaler.fit_transform(X)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_minmax, y, test_size=0.3, random_state=1)

In [18]:
rf = RandomForestClassifier(n_estimators=50, max_depth=5)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=50)

In [19]:
y_pred = rf.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.36      0.48      2747
           1       0.60      0.87      0.71      2996
           2       1.00      0.96      0.98       275

    accuracy                           0.64      6018
   macro avg       0.77      0.73      0.72      6018
weighted avg       0.67      0.64      0.62      6018

